## DB 2-stage 실험1

- Reranker = "BAAI/bge-reranker-v2-m3"

1. Faiss로 100개 -> ElasticSearch로 20개 -> Reranker로 Top 5 -> LLM에 평가
2. ElasticSearch로 100개 -> Faiss로 20개 -> Reranker로 Top 5 -> LLM에 평가


- QnA 및 GT는 nid: 171번을 기반으로 만듦

    - Q: DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?

    - Ground Truth Context nid: 171번

In [3]:
# [셀 1] - 필요한 라이브러리 임포트
import requests
from sentence_transformers import SentenceTransformer, CrossEncoder
from elasticsearch import Elasticsearch
import numpy as np
from typing import List, Dict
from langchain_community.chat_models import ChatOpenAI
import os

In [11]:
# [셀 2] - 기본 설정
from dotenv import load_dotenv

# 디렉토리 변경 (작업자의 디렉토리로 변경 필요)
# os.chdir('/data/ephemeral/home/Task4')

# .env 로드
load_dotenv()

# API 키 및 URL 설정
FAISS_API_KEY = os.getenv("FAISS_API_KEY")
FAISS_URL = "http://3.34.62.202:8060"
ES_HOST = "http://3.34.62.202:9200"

# 세션 및 클라이언트 초기화
session = requests.Session()
es = Elasticsearch(ES_HOST)

# 모델 초기화
embedding_model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')
reranker = CrossEncoder('BAAI/bge-reranker-v2-m3')

In [13]:
# [셀 3] - 실험 1: Faiss -> ES -> Reranker
def experiment_1(query: str):
    print("=== 실험 1: Faiss -> ES -> Reranker ===\n")
    
    # 1. FAISS 검색 (100개)
    print("1. FAISS 검색 (100개 문서)")
    query_vector = embedding_model.encode(query).tolist()
    faiss_response = session.post(
        f"{FAISS_URL}/api/context/search-by-vector",
        headers={"x-api-key": FAISS_API_KEY},
        json={
            "index": "prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
            "query_vector": query_vector,
            "size": 100
        },
        timeout=15
    )
    faiss_results = faiss_response.json()['results']
    print(f"FAISS 검색 결과 수: {len(faiss_results)}\n")
    
    # 2. Elasticsearch 검색 (20개)
    print("2. Elasticsearch 필터링 (20개 문서)")
    nid_values = [item['document_id']+1 for item in faiss_results]
    es_query = {
        "query": {
            "bool": {
                "filter": {
                    "terms": {
                        "nid": nid_values
                    }
                },
                "must": {
                    "match": {
                        "text": query
                    }
                }
            }
        },
        "size": 20
    }
    es_response = es.search(
        index="prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
        body=es_query
    )
    
    # ES 결과 문서와 메타데이터 저장
    documents = []
    for hit in es_response["hits"]["hits"]:
        metadata = hit["_source"]["metadata"].copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
        
        documents.append({
            'text': hit["_source"]["text"],
            'metadata': {
                **metadata,
                'es_score': hit["_score"]
            }
        })
    print(f"Elasticsearch 필터링 결과 수: {len(documents)}\n")
    
    # 3. Reranker (Top 5)
    print("3. Reranker로 Top 5 선정")
    pairs = [[query, doc['text']] for doc in documents]
    scores = reranker.predict(pairs)
    
    # 스코어와 함께 문서 정렬
    for doc, score in zip(documents, scores):
        doc['metadata']['reranker_score'] = score
    
    ranked_results = sorted(documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

# 실험 1 실행
test_query = "DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?"
results_1 = experiment_1(test_query)

=== 실험 1: Faiss -> ES -> Reranker ===

1. FAISS 검색 (100개 문서)
FAISS 검색 결과 수: 100

2. Elasticsearch 필터링 (20개 문서)
Elasticsearch 필터링 결과 수: 20

3. Reranker로 Top 5 선정

=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요<br>부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대<br>한다. 2025년 영업이익은 36조원으로 전망한다

In [14]:
# [셀 4] - 실험 2: ES -> Faiss -> Reranker
def experiment_2(query: str):
    print("=== 실험 2: ES -> Faiss -> Reranker ===\n")
    
    # 1. Elasticsearch 검색 (100개)
    print("1. Elasticsearch 검색 (100개 문서)")
    es_query = {
        "query": {
            "match": {
                "text": query
            }
        },
        "size": 100
    }
    es_response = es.search(
        index="prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
        body=es_query
    )
    
    documents = []
    for hit in es_response["hits"]["hits"]:
        metadata = hit["_source"]["metadata"].copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
        
        documents.append({
            'text': hit["_source"]["text"],
            'metadata': {
                **metadata,
                'es_score': hit["_score"]
            }
        })
    print(f"Elasticsearch 검색 결과 수: {len(documents)}\n")
    
    # 2. FAISS 유사도 계산 (20개)
    print("2. FAISS 유사도 계산으로 필터링 (20개 문서)")
    texts = [doc['text'] for doc in documents]
    vectors = embedding_model.encode(texts).tolist()
    query_vector = embedding_model.encode(query).tolist()
    
    # 유사도 계산 및 상위 20개 선택
    similarities = np.dot(vectors, query_vector) / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query_vector))
    top_20_indices = np.argsort(similarities)[-20:][::-1]
    
    filtered_documents = [documents[i] for i in top_20_indices]
    for doc, sim in zip(filtered_documents, similarities[top_20_indices]):
        doc['metadata']['faiss_score'] = float(sim)
    print(f"FAISS 필터링 결과 수: {len(filtered_documents)}\n")
    
    # 3. Reranker (Top 5)
    print("3. Reranker로 Top 5 선정")
    pairs = [[query, doc['text']] for doc in filtered_documents]
    scores = reranker.predict(pairs)
    
    # 스코어 추가 및 정렬
    for doc, score in zip(filtered_documents, scores):
        doc['metadata']['reranker_score'] = score
    
    ranked_results = sorted(filtered_documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

# 실험 2 실행
results_2 = experiment_2(test_query)

=== 실험 2: ES -> Faiss -> Reranker ===

1. Elasticsearch 검색 (100개 문서)
Elasticsearch 검색 결과 수: 100

2. FAISS 유사도 계산으로 필터링 (20개 문서)
FAISS 필터링 결과 수: 20

3. Reranker로 Top 5 선정

=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요<br>부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대<br>한다. 2025년 영업이익은 36

## 결과
- 두 방식 모두 목표 문서를 가장 높은 순위로 찾음
- 단일 케이스로는 성능 차이를 판단하기 어려움

---

## 추가 실험2 진행
- 다양한 난이도의 QnA 쌍으로 테스트
- 각 방식의 처리 시간 비교(검색 속도, 시스템 리소스 사용량)
- TOP1 뿐만 아니라 TOP5까지의 결과 관련성 비교

In [15]:
# [셀 1] - 테스트 케이스 및 평가 도구 설정
import time
from typing import Dict, List, Any
import statistics

test_queries = [
    {
        "query": "DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?",
        "target_nid": 171,
        "difficulty": "easy"
    },
    {
        "query": "SK하이닉스에 대한 DS증권의 가장 최근 목표주가와 향후 2년간의 영업이익 전망은?",
        "target_nid": 171,
        "difficulty": "medium"
    },
    {
        "query": "메모리 반도체 업체 중 SK하이닉스의 향후 실적 전망은 어떠한가요? 특히 2024-2025년의 영업이익 추정치가 궁금합니다.",
        "target_nid": 171,
        "difficulty": "hard"
    }
]

def evaluate_search_results(results: List[Dict], target_nid: int) -> Dict[str, Any]:
    """검색 결과 평가"""
    evaluation = {}
    
    # MRR 계산
    mrr = 0
    for i, doc in enumerate(results, 1):
        if doc['metadata'].get('nid') == target_nid:
            mrr = 1 / i
            evaluation['rank'] = i
            break
    evaluation['mrr'] = mrr
    
    # TOP5 관련성 (target_nid가 포함되어 있는지)
    evaluation['in_top5'] = any(doc['metadata'].get('nid') == target_nid for doc in results)
    
    return evaluation

In [16]:
# [셀 2] - 실험 실행 함수
def run_experiments(test_queries: List[Dict]):
    results = {
        "faiss_first": {"mrr": [], "processing_time": [], "in_top5": [], "ranks": []},
        "es_first": {"mrr": [], "processing_time": [], "in_top5": [], "ranks": []}
    }
    
    for query_data in test_queries:
        print(f"\n테스트 케이스: {query_data['difficulty']}")
        print(f"Query: {query_data['query']}")
        
        # Faiss -> ES -> Reranker
        start_time = time.time()
        faiss_results = experiment_1(query_data['query'])
        faiss_time = time.time() - start_time
        faiss_eval = evaluate_search_results(faiss_results, query_data['target_nid'])
        
        # ES -> Faiss -> Reranker
        start_time = time.time()
        es_results = experiment_2(query_data['query'])
        es_time = time.time() - start_time
        es_eval = evaluate_search_results(es_results, query_data['target_nid'])
        
        # 결과 저장
        results["faiss_first"]["mrr"].append(faiss_eval["mrr"])
        results["faiss_first"]["processing_time"].append(faiss_time)
        results["faiss_first"]["in_top5"].append(faiss_eval["in_top5"])
        results["faiss_first"]["ranks"].append(faiss_eval.get("rank", 6))
        
        results["es_first"]["mrr"].append(es_eval["mrr"])
        results["es_first"]["processing_time"].append(es_time)
        results["es_first"]["in_top5"].append(es_eval["in_top5"])
        results["es_first"]["ranks"].append(es_eval.get("rank", 6))
        
        print("\n결과 비교:")
        print(f"Faiss First - Rank: {faiss_eval.get('rank', 'Not found')}, Time: {faiss_time:.2f}s")
        print(f"ES First - Rank: {es_eval.get('rank', 'Not found')}, Time: {es_time:.2f}s")
    
    return results

In [17]:
# [셀 3] - 결과 분석
def analyze_results(results: Dict):
    print("\n=== 종합 평가 결과 ===")
    
    methods = ["faiss_first", "es_first"]
    for method in methods:
        print(f"\n{method.replace('_', ' ').title()}:")
        print(f"평균 MRR: {statistics.mean(results[method]['mrr']):.4f}")
        print(f"평균 처리 시간: {statistics.mean(results[method]['processing_time']):.2f}초")
        print(f"TOP5 포함 비율: {sum(results[method]['in_top5'])/len(results[method]['in_top5'])*100:.1f}%")
        print(f"평균 순위: {statistics.mean(results[method]['ranks']):.2f}")
        
    # 방식 선택을 위한 종합 점수 계산
    scores = {}
    for method in methods:
        mrr_score = statistics.mean(results[method]['mrr'])
        time_score = 1 / statistics.mean(results[method]['processing_time'])  # 처리시간이 짧을수록 높은 점수
        top5_score = sum(results[method]['in_top5'])/len(results[method]['in_top5'])
        
        # 종합 점수 (가중치는 조정 가능)
        scores[method] = (mrr_score * 0.4 + time_score * 0.3 + top5_score * 0.3)
    
    print("\n=== 최종 추천 ===")
    recommended = max(scores.items(), key=lambda x: x[1])[0]
    print(f"추천 방식: {recommended.replace('_', ' ').title()}")
    print("추천 이유:")
    for method, score in scores.items():
        print(f"- {method.replace('_', ' ').title()}: {score:.4f}")

In [18]:
# [셀 4] - 실험 실행
results = run_experiments(test_queries)
analyze_results(results)


테스트 케이스: easy
Query: DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?
=== 실험 1: Faiss -> ES -> Reranker ===

1. FAISS 검색 (100개 문서)
FAISS 검색 결과 수: 100

2. Elasticsearch 필터링 (20개 문서)
Elasticsearch 필터링 결과 수: 20

3. Reranker로 Top 5 선정

=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이

- FAISS First가 더 적합해보이는 이유

    - 금융 문서는 숫자, 재무 지표, 전망치 등 구조화된 정보가 많음
    - 이런 정보는 문맥상 의미적 유사성이 중요 (예: "영업이익", "매출액", "목표가" 등)
    - FAISS의 벡터 검색이 이러한 의미적 관계를 잘 포착할 수 있음
    - ES의 키워드 매칭은 두 번째 단계에서 정확도를 보완하는 역할


- 일반적인 검색 방식?

    - 일반적으로는 ES를 먼저 사용하는 것이 더 일반적
    - 그 이유는 키워드 매칭이 계산 비용이 적고, 명확한 키워드가 있는 경우 정확도가 높음
- 하지만 금융 도메인은 다를 수 있음
    - 유사한 표현이 많음 (예: "실적 전망", "영업이익 추정치", "실적 예상")
    - 숫자와 함께 나오는 문맥이 중요 (예: "2024년 영업이익 24조 원")
    - 시계열적 관계 이해 필요 (과거 실적 vs 미래 전망)

- 요약:
    - 의미적 유사성 우선 검색
    - 키워드 매칭으로 정확도 보완
    - Reranker로 최종 순위 조정

------

## + ChromaDB와의 비교 실험3

In [20]:
# [셀 1] - 필요한 라이브러리 임포트
# aistages 가상환경에서의 conda python version 문제로 아래처럼 특정 라이브러리 버전을 강제 지정

import os
os.environ['LD_PRELOAD'] = '/lib/x86_64-linux-gnu/libsqlite3.so.0'
os.environ['LD_LIBRARY_PATH'] = '/lib/x86_64-linux-gnu'

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

import requests
from sentence_transformers import SentenceTransformer, CrossEncoder
from elasticsearch import Elasticsearch
import numpy as np
from typing import List, Dict, Any
import time
import statistics
from dotenv import load_dotenv
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.embeddings.base import Embeddings

# 환경변수 로드
load_dotenv()

True

In [21]:
# [셀 2] - 기본 설정 및 모델 초기화
# API 키 및 URL 설정
FAISS_API_KEY = os.getenv("FAISS_API_KEY")
FAISS_URL = "http://3.34.62.202:8060"
ES_HOST = "http://3.34.62.202:9200"

# 세션 및 클라이언트 초기화
session = requests.Session()
es = Elasticsearch(ES_HOST)

# ChromaDB 클라이언트 초기화
persist_directory = os.path.abspath("./chroma.db")
chroma_client = chromadb.Client(Settings(
    persist_directory=persist_directory,
    is_persistent=True,
    anonymized_telemetry=False
))

# 임베딩 모델 초기화
class CustomSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts)
        return embeddings.tolist()

    def embed_query(self, text: str) -> List[float]:
        embedding = self.model.encode(text)
        return embedding.tolist()

embedding_model = CustomSentenceTransformerEmbeddings('intfloat/multilingual-e5-large-instruct')
reranker = CrossEncoder('BAAI/bge-reranker-v2-m3')

Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'


In [27]:
# [셀 3] - 실험 함수 구현
def experiment_1(query: str):
    """FAISS -> ES -> Reranker 실험"""
    print("=== 실험 1: Faiss -> ES -> Reranker ===\n")
    
    # 1. FAISS 검색 (100개)
    print("1. FAISS 검색 (100개 문서)")
    query_vector = embedding_model.embed_query(query)
    faiss_response = session.post(
        f"{FAISS_URL}/api/context/search-by-vector",
        headers={"x-api-key": FAISS_API_KEY},
        json={
            "index": "prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
            "query_vector": query_vector,
            "size": 100
        },
        timeout=15
    )
    faiss_results = faiss_response.json()['results']
    print(f"FAISS 검색 결과 수: {len(faiss_results)}\n")
    
    # 2. Elasticsearch 검색 (20개)
    print("2. Elasticsearch 필터링 (20개 문서)")
    nid_values = [item['document_id']+1 for item in faiss_results]
    es_query = {
        "query": {
            "bool": {
                "filter": {
                    "terms": {
                        "nid": nid_values
                    }
                },
                "must": {
                    "match": {
                        "text": query
                    }
                }
            }
        },
        "size": 20
    }
    es_response = es.search(
        index="prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
        body=es_query
    )
    
    # ES 결과 문서와 메타데이터 저장
    documents = []
    for hit in es_response["hits"]["hits"]:
        metadata = hit["_source"]["metadata"].copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
        
        documents.append({
            'text': hit["_source"]["text"],
            'metadata': {
                **metadata,
                'es_score': hit["_score"]
            }
        })
    print(f"Elasticsearch 필터링 결과 수: {len(documents)}\n")
    
    # 3. Reranker (Top 5)
    print("3. Reranker로 Top 5 선정")
    pairs = [[query, doc['text']] for doc in documents]
    scores = reranker.predict(pairs)
    
    for doc, score in zip(documents, scores):
        doc['metadata']['reranker_score'] = score
    
    ranked_results = sorted(documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

In [28]:
# [셀 4] - ES First 실험
def experiment_2(query: str):
    """ES -> FAISS -> Reranker 실험"""
    print("=== 실험 2: ES -> Faiss -> Reranker ===\n")
    
    # 1. Elasticsearch 검색 (100개)
    print("1. Elasticsearch 검색 (100개 문서)")
    es_query = {
        "query": {
            "match": {
                "text": query
            }
        },
        "size": 100
    }
    es_response = es.search(
        index="prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
        body=es_query
    )
    
    documents = []
    for hit in es_response["hits"]["hits"]:
        metadata = hit["_source"]["metadata"].copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
        
        documents.append({
            'text': hit["_source"]["text"],
            'metadata': {
                **metadata,
                'es_score': hit["_score"]
            }
        })
    print(f"Elasticsearch 검색 결과 수: {len(documents)}\n")
    
    # 2. FAISS 유사도 계산 (20개)
    print("2. FAISS 유사도 계산으로 필터링 (20개 문서)")
    texts = [doc['text'] for doc in documents]
    vectors = embedding_model.embed_documents(texts)
    query_vector = embedding_model.embed_query(query)
    
    similarities = np.dot(vectors, query_vector) / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query_vector))
    top_20_indices = np.argsort(similarities)[-20:][::-1]
    
    filtered_documents = [documents[i] for i in top_20_indices]
    for doc, sim in zip(filtered_documents, similarities[top_20_indices]):
        doc['metadata']['faiss_score'] = float(sim)
    print(f"FAISS 필터링 결과 수: {len(filtered_documents)}\n")
    
    # 3. Reranker (Top 5)
    print("3. Reranker로 Top 5 선정")
    pairs = [[query, doc['text']] for doc in filtered_documents]
    scores = reranker.predict(pairs)
    
    for doc, score in zip(filtered_documents, scores):
        doc['metadata']['reranker_score'] = score
    
    ranked_results = sorted(filtered_documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

In [29]:
# [셀 5] - ChromaDB 실험
def experiment_3(query: str):
    """ChromaDB -> Reranker 실험"""
    print("=== 실험 3: ChromaDB -> Reranker ===\n")
    
    # 1. ChromaDB 검색 (100개)
    print("1. ChromaDB 검색 (100개 문서)")
    langchain_chroma = Chroma(
        client=chroma_client,
        collection_name="financial_reports",
        embedding_function=embedding_model
    )
    base_retriever = langchain_chroma.as_retriever(search_kwargs={"k": 100})
    initial_docs = base_retriever.get_relevant_documents(query)
    print(f"ChromaDB 검색 결과 수: {len(initial_docs)}\n")
    
    # 2. Reranker (Top 5)
    print("2. Reranker로 Top 5 선정")
    pairs = [[query, doc.page_content] for doc in initial_docs]
    scores = reranker.predict(pairs)
    
    documents = []
    for doc, score in zip(initial_docs, scores):
        metadata = doc.metadata.copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
            
        documents.append({
            'text': doc.page_content,
            'metadata': {
                **metadata,
                'chroma_score': doc.metadata.get('distance', 0),
                'reranker_score': score
            }
        })
    
    ranked_results = sorted(documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

In [30]:
# [셀 6] - 평가 함수 및 실험 실행
def evaluate_search_results(results: List[Dict], target_nid: int) -> Dict[str, Any]:
   """검색 결과 평가"""
   evaluation = {}
   
   # MRR 계산
   mrr = 0
   for i, doc in enumerate(results, 1):
       if doc['metadata'].get('nid') == target_nid:
           mrr = 1 / i
           evaluation['rank'] = i
           break
   evaluation['mrr'] = mrr
   
   # TOP5 관련성
   evaluation['in_top5'] = any(doc['metadata'].get('nid') == target_nid for doc in results)
   
   return evaluation

def run_experiments(test_queries: List[Dict]):
   """실험 실행 및 결과 수집"""
   results = {
       "faiss_first": {"mrr": [], "processing_time": [], "in_top5": [], "ranks": []},
       "es_first": {"mrr": [], "processing_time": [], "in_top5": [], "ranks": []},
       "chroma": {"mrr": [], "processing_time": [], "in_top5": [], "ranks": []}
   }
   
   for query_data in test_queries:
       print(f"\n테스트 케이스: {query_data['difficulty']}")
       print(f"Query: {query_data['query']}")
       
       for method, experiment_func in [
           ("faiss_first", experiment_1),
           ("es_first", experiment_2),
           ("chroma", experiment_3)
       ]:
           start_time = time.time()
           method_results = experiment_func(query_data['query'])
           method_time = time.time() - start_time
           method_eval = evaluate_search_results(method_results, query_data['target_nid'])
           
           results[method]["mrr"].append(method_eval["mrr"])
           results[method]["processing_time"].append(method_time)
           results[method]["in_top5"].append(method_eval["in_top5"])
           results[method]["ranks"].append(method_eval.get("rank", 6))
           
           print(f"\n{method} - Rank: {method_eval.get('rank', 'Not found')}, Time: {method_time:.2f}s")
   
   return results

def analyze_results(results: Dict):
   """실험 결과 분석"""
   print("\n=== 종합 평가 결과 ===")
   
   for method in results.keys():
       print(f"\n{method.replace('_', ' ').title()}:")
       print(f"평균 MRR: {statistics.mean(results[method]['mrr']):.4f}")
       print(f"평균 처리 시간: {statistics.mean(results[method]['processing_time']):.2f}초")
       print(f"TOP5 포함 비율: {sum(results[method]['in_top5'])/len(results[method]['in_top5'])*100:.1f}%")
       print(f"평균 순위: {statistics.mean(results[method]['ranks']):.2f}")
   
   # 종합 점수 계산
   scores = {}
   for method in results.keys():
       mrr_score = statistics.mean(results[method]['mrr'])
       time_score = 1 / statistics.mean(results[method]['processing_time'])
       top5_score = sum(results[method]['in_top5'])/len(results[method]['in_top5'])
       
       scores[method] = (mrr_score * 0.4 + time_score * 0.3 + top5_score * 0.3)
   
   print("\n=== 최종 추천 ===")
   recommended = max(scores.items(), key=lambda x: x[1])[0]
   print(f"추천 방식: {recommended.replace('_', ' ').title()}")
   print("\n추천 이유:")
   for method, score in scores.items():
       print(f"- {method.replace('_', ' ').title()}: {score:.4f}")

In [31]:
# [셀 7] - 테스트 실행
test_queries = [
    {
        "query": "DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?",
        "target_nid": 171,
        "difficulty": "easy"
    },
    {
        "query": "SK하이닉스에 대한 DS증권의 가장 최근 목표주가와 향후 2년간의 영업이익 전망은?",
        "target_nid": 171,
        "difficulty": "medium"
    },
    {
        "query": "메모리 반도체 업체 중 SK하이닉스의 향후 실적 전망은 어떠한가요? 특히 2024-2025년의 영업이익 추정치가 궁금합니다.",
        "target_nid": 171,
        "difficulty": "hard"
    }
]

# 실험 실행
results = run_experiments(test_queries)
analyze_results(results)


테스트 케이스: easy
Query: DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?
=== 실험 1: Faiss -> ES -> Reranker ===

1. FAISS 검색 (100개 문서)
FAISS 검색 결과 수: 100

2. Elasticsearch 필터링 (20개 문서)
Elasticsearch 필터링 결과 수: 20

3. Reranker로 Top 5 선정

=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이

/tmp/ipykernel_685044/3430306030.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  langchain_chroma = Chroma(
Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
/tmp/ipykernel_685044/3430306030.py:14: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  initial_docs = base_retriever.get_relevant_documents(query)
Failed to send telemetry event CollectionQueryEvent: module 'chromadb' has no attribute 'get_settings'



=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요<br>부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대<br>한다. 2025년 영업이익은 36조원으로 전망한다.</p>"}
- uuid: cf2e2bb8-5d26-4f46-8650-e330d0339abd
- nid: 171
- has_coordinates: True
- es_score: 24.7488
- faiss_score: 0.9458
- reranker_score: 0.99997925758

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'



=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요<br>부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대<br>한다. 2025년 영업이익은 36조원으로 전망한다.</p>"}
- uuid: cf2e2bb8-5d26-4f46-8650-e330d0339abd
- nid: 171
- has_coordinates: True
- es_score: 7.1608
- faiss_score: 0.9147
- reranker_score: 0.999494433403

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'



=== Top 5 Documents ===

[Document 1]
Metadata:
- category: list
- page: 1
- id: 2
- company_name: SK하이닉스
- report_date: 20241209
- securities_firm: 유진투자증권
- source_file: SK하이닉스_20241209_유진투자증권.pdf
- content_types: {'markdown': '- ◼ 최근 일련의 상황들을 보면 코리아 디스카운트의 가장 큰 요인은 어쩌면 대주주 리스크와 정치 지도자 리스크였을지도 모른\n- 다는 생각이 든다. 트럼프의 재등장으로 글로벌 지경학은 밀림의 한 가운데 들어섰다. 거기에 더해 반도체 업황은 둔화하고\n- 있으며, 수출 통제 등 부담까지 더해지고 있다. 하지만 이런 절체절명의 상황에서 우리나라는 계엄 발동과 해제, 그리고\n- 지도자 공백이라는 초현실적 상황을 맞이하게 되었다. 이와 같은 국내외적 리스크를 감안할 때 주요 기업들의 실적 전망\n- 하향과 밸류에이션 조정이 불가피해 보인다.\n- ◼ IT 수요 부진과 메모리 재고조정 영향, 반도체 규제 등을 반영해 SK하이닉스의 실적을 다음과 같이 조정한다. 4분기\n- 실적은 매출 18.4조원, 영업이익 7.3조원을 예상한다. 인상적인 기술력을 감안할 때 2025년 실적은 매출 70.4조원\n- 영업이익 25.6조원으로 성장세를 이어갈 가능성은 있지만, 현재 시장 기대는 다소 높아 보인다.\n- ◼ 밸류업을 가로막는 코리아 리스크로 타겟 밸류에이션을 P/B 2.0에서 1.8배로 조정하고, 목표주가를 220,000원으로\n- 하향한다. 다만, 현주가는 P/B 1.4배로 타겟 대비 현저히 낮다는 점에서 투자의견은 ‘BUY’를 유지한다.\n', 'text': '◼ 최근 일련의 상황들을 보면 코리아 디스카운트의 가장 큰 요인은 어쩌면 대주주 리스크와 정치 지도자 리스크였을지도 모른\n다는 생각이 든다. 트럼프의 재등장으로 글로벌 지경학은 밀림의 한 가운데 들어섰다.

## 결과

- ChromaDB가 성능이 높지만 처리 속도가 3-4배 차이가 나고 있음
- 성능 + 속도 등을 고려해서 문서 검색 수를 변경하여 다시 실험해볼 필요가 있을 것 같음
    - 문서 검색 수를 변경해서 Faiss + ES 검색 방식의 정확도가 더 높아질 수 있기 때문에 

----
## 기존에 가져오는 문서 개수를 변경 실험4

### 기존
- 처음에 100개 -> 두번째 -> 20개 -> reranker 5개


### 변경
- 처음에 100개 -> 두번째 40개 -> reranker 5개

In [32]:
# [셀 3] - 실험 함수 구현
def experiment_1(query: str):
    """FAISS -> ES -> Reranker 실험"""
    print("=== 실험 1: Faiss -> ES -> Reranker ===\n")
    
    # 1. FAISS 검색 (100개)
    print("1. FAISS 검색 (100개 문서)")
    query_vector = embedding_model.embed_query(query)
    faiss_response = session.post(
        f"{FAISS_URL}/api/context/search-by-vector",
        headers={"x-api-key": FAISS_API_KEY},
        json={
            "index": "prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
            "query_vector": query_vector,
            "size": 100
        },
        timeout=15
    )
    faiss_results = faiss_response.json()['results']
    print(f"FAISS 검색 결과 수: {len(faiss_results)}\n")
    
    # 2. Elasticsearch 검색 (40개)
    print("2. Elasticsearch 필터링 (40개 문서)")
    nid_values = [item['document_id']+1 for item in faiss_results]
    es_query = {
        "query": {
            "bool": {
                "filter": {
                    "terms": {
                        "nid": nid_values
                    }
                },
                "must": {
                    "match": {
                        "text": query
                    }
                }
            }
        },
        "size": 40
    }
    es_response = es.search(
        index="prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
        body=es_query
    )
    
    # ES 결과 문서와 메타데이터 저장
    documents = []
    for hit in es_response["hits"]["hits"]:
        metadata = hit["_source"]["metadata"].copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
        
        documents.append({
            'text': hit["_source"]["text"],
            'metadata': {
                **metadata,
                'es_score': hit["_score"]
            }
        })
    print(f"Elasticsearch 필터링 결과 수: {len(documents)}\n")
    
    # 3. Reranker (Top 5)
    print("3. Reranker로 Top 5 선정")
    pairs = [[query, doc['text']] for doc in documents]
    scores = reranker.predict(pairs)
    
    for doc, score in zip(documents, scores):
        doc['metadata']['reranker_score'] = score
    
    ranked_results = sorted(documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

In [36]:
# [셀 4] - ES First 실험
def experiment_2(query: str):
    """ES -> FAISS -> Reranker 실험"""
    print("=== 실험 2: ES -> Faiss -> Reranker ===\n")
    
    # 1. Elasticsearch 검색 (100개)
    print("1. Elasticsearch 검색 (100개 문서)")
    es_query = {
        "query": {
            "match": {
                "text": query
            }
        },
        "size": 100
    }
    es_response = es.search(
        index="prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
        body=es_query
    )
    
    documents = []
    for hit in es_response["hits"]["hits"]:
        metadata = hit["_source"]["metadata"].copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
        
        documents.append({
            'text': hit["_source"]["text"],
            'metadata': {
                **metadata,
                'es_score': hit["_score"]
            }
        })
    print(f"Elasticsearch 검색 결과 수: {len(documents)}\n")
    
    # 2. FAISS 유사도 계산 (40개)
    print("2. FAISS 유사도 계산으로 필터링 (40개 문서)")
    texts = [doc['text'] for doc in documents]
    vectors = embedding_model.embed_documents(texts)
    query_vector = embedding_model.embed_query(query)
    
    similarities = np.dot(vectors, query_vector) / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query_vector))
    top_40_indices = np.argsort(similarities)[-40:][::-1]
    
    filtered_documents = [documents[i] for i in top_40_indices]
    for doc, sim in zip(filtered_documents, similarities[top_40_indices]):
        doc['metadata']['faiss_score'] = float(sim)
    print(f"FAISS 필터링 결과 수: {len(filtered_documents)}\n")
    
    # 3. Reranker (Top 5)
    print("3. Reranker로 Top 5 선정")
    pairs = [[query, doc['text']] for doc in filtered_documents]
    scores = reranker.predict(pairs)
    
    for doc, score in zip(filtered_documents, scores):
        doc['metadata']['reranker_score'] = score
    
    ranked_results = sorted(filtered_documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

In [34]:
# [셀 5] - ChromaDB 실험
def experiment_3(query: str):
    """ChromaDB -> Reranker 실험"""
    print("=== 실험 3: ChromaDB -> Reranker ===\n")
    
    # 1. ChromaDB 검색 (100개)
    print("1. ChromaDB 검색 (100개 문서)")
    langchain_chroma = Chroma(
        client=chroma_client,
        collection_name="financial_reports",
        embedding_function=embedding_model
    )
    base_retriever = langchain_chroma.as_retriever(search_kwargs={"k": 100})
    initial_docs = base_retriever.get_relevant_documents(query)
    print(f"ChromaDB 검색 결과 수: {len(initial_docs)}\n")
    
    # 2. Reranker (Top 5)
    print("2. Reranker로 Top 5 선정")
    pairs = [[query, doc.page_content] for doc in initial_docs]
    scores = reranker.predict(pairs)
    
    documents = []
    for doc, score in zip(initial_docs, scores):
        metadata = doc.metadata.copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
            
        documents.append({
            'text': doc.page_content,
            'metadata': {
                **metadata,
                'chroma_score': doc.metadata.get('distance', 0),
                'reranker_score': score
            }
        })
    
    ranked_results = sorted(documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

In [37]:
# [셀 6]은 기존거 사용
# [셀 7] - 테스트 실행
test_queries = [
    {
        "query": "DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?",
        "target_nid": 171,
        "difficulty": "easy"
    },
    {
        "query": "SK하이닉스에 대한 DS증권의 가장 최근 목표주가와 향후 2년간의 영업이익 전망은?",
        "target_nid": 171,
        "difficulty": "medium"
    },
    {
        "query": "메모리 반도체 업체 중 SK하이닉스의 향후 실적 전망은 어떠한가요? 특히 2024-2025년의 영업이익 추정치가 궁금합니다.",
        "target_nid": 171,
        "difficulty": "hard"
    }
]

# 실험 실행
results = run_experiments(test_queries)
analyze_results(results)


테스트 케이스: easy
Query: DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?
=== 실험 1: Faiss -> ES -> Reranker ===

1. FAISS 검색 (100개 문서)
FAISS 검색 결과 수: 100

2. Elasticsearch 필터링 (20개 문서)
Elasticsearch 필터링 결과 수: 40

3. Reranker로 Top 5 선정

=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'



=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요<br>부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대<br>한다. 2025년 영업이익은 36조원으로 전망한다.</p>"}
- uuid: cf2e2bb8-5d26-4f46-8650-e330d0339abd
- nid: 171
- has_coordinates: True
- es_score: 24.7488
- faiss_score: 0.9458
- reranker_score: 0.99997925758

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'



=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요<br>부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대<br>한다. 2025년 영업이익은 36조원으로 전망한다.</p>"}
- uuid: cf2e2bb8-5d26-4f46-8650-e330d0339abd
- nid: 171
- has_coordinates: True
- es_score: 7.1608
- faiss_score: 0.9147
- reranker_score: 0.999494433403

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'



=== Top 5 Documents ===

[Document 1]
Metadata:
- category: list
- page: 1
- id: 2
- company_name: SK하이닉스
- report_date: 20241209
- securities_firm: 유진투자증권
- source_file: SK하이닉스_20241209_유진투자증권.pdf
- content_types: {'markdown': '- ◼ 최근 일련의 상황들을 보면 코리아 디스카운트의 가장 큰 요인은 어쩌면 대주주 리스크와 정치 지도자 리스크였을지도 모른\n- 다는 생각이 든다. 트럼프의 재등장으로 글로벌 지경학은 밀림의 한 가운데 들어섰다. 거기에 더해 반도체 업황은 둔화하고\n- 있으며, 수출 통제 등 부담까지 더해지고 있다. 하지만 이런 절체절명의 상황에서 우리나라는 계엄 발동과 해제, 그리고\n- 지도자 공백이라는 초현실적 상황을 맞이하게 되었다. 이와 같은 국내외적 리스크를 감안할 때 주요 기업들의 실적 전망\n- 하향과 밸류에이션 조정이 불가피해 보인다.\n- ◼ IT 수요 부진과 메모리 재고조정 영향, 반도체 규제 등을 반영해 SK하이닉스의 실적을 다음과 같이 조정한다. 4분기\n- 실적은 매출 18.4조원, 영업이익 7.3조원을 예상한다. 인상적인 기술력을 감안할 때 2025년 실적은 매출 70.4조원\n- 영업이익 25.6조원으로 성장세를 이어갈 가능성은 있지만, 현재 시장 기대는 다소 높아 보인다.\n- ◼ 밸류업을 가로막는 코리아 리스크로 타겟 밸류에이션을 P/B 2.0에서 1.8배로 조정하고, 목표주가를 220,000원으로\n- 하향한다. 다만, 현주가는 P/B 1.4배로 타겟 대비 현저히 낮다는 점에서 투자의견은 ‘BUY’를 유지한다.\n', 'text': '◼ 최근 일련의 상황들을 보면 코리아 디스카운트의 가장 큰 요인은 어쩌면 대주주 리스크와 정치 지도자 리스크였을지도 모른\n다는 생각이 든다. 트럼프의 재등장으로 글로벌 지경학은 밀림의 한 가운데 들어섰다.

## 결과

- 현재 상황 분석:

    - 첫 단계(100개): 충분히 넓은 후보군 확보
    - 두 번째 단계(40개): 성능 저하 발생
    - 최종 단계(5개): Reranker로 최종 선택


- 성능 저하 가능한 원인:
    - Reranker에 많은 문서를 전달 -> 처리 속도 증가

----

## 실험5

- 100 개 -> 15 개 -> 5개

In [38]:
# [셀 3] - 실험 함수 구현
def experiment_1(query: str):
    """FAISS -> ES -> Reranker 실험"""
    print("=== 실험 1: Faiss -> ES -> Reranker ===\n")
    
    # 1. FAISS 검색 (100개)
    print("1. FAISS 검색 (100개 문서)")
    query_vector = embedding_model.embed_query(query)
    faiss_response = session.post(
        f"{FAISS_URL}/api/context/search-by-vector",
        headers={"x-api-key": FAISS_API_KEY},
        json={
            "index": "prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
            "query_vector": query_vector,
            "size": 100
        },
        timeout=15
    )
    faiss_results = faiss_response.json()['results']
    print(f"FAISS 검색 결과 수: {len(faiss_results)}\n")
    
    # 2. Elasticsearch 검색 (15개)
    print("2. Elasticsearch 필터링 (15개 문서)")
    nid_values = [item['document_id']+1 for item in faiss_results]
    es_query = {
        "query": {
            "bool": {
                "filter": {
                    "terms": {
                        "nid": nid_values
                    }
                },
                "must": {
                    "match": {
                        "text": query
                    }
                }
            }
        },
        "size": 15
    }
    es_response = es.search(
        index="prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
        body=es_query
    )
    
    # ES 결과 문서와 메타데이터 저장
    documents = []
    for hit in es_response["hits"]["hits"]:
        metadata = hit["_source"]["metadata"].copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
        
        documents.append({
            'text': hit["_source"]["text"],
            'metadata': {
                **metadata,
                'es_score': hit["_score"]
            }
        })
    print(f"Elasticsearch 필터링 결과 수: {len(documents)}\n")
    
    # 3. Reranker (Top 5)
    print("3. Reranker로 Top 5 선정")
    pairs = [[query, doc['text']] for doc in documents]
    scores = reranker.predict(pairs)
    
    for doc, score in zip(documents, scores):
        doc['metadata']['reranker_score'] = score
    
    ranked_results = sorted(documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

In [39]:
# [셀 4] - ES First 실험
def experiment_2(query: str):
    """ES -> FAISS -> Reranker 실험"""
    print("=== 실험 2: ES -> Faiss -> Reranker ===\n")
    
    # 1. Elasticsearch 검색 (100개)
    print("1. Elasticsearch 검색 (100개 문서)")
    es_query = {
        "query": {
            "match": {
                "text": query
            }
        },
        "size": 100
    }
    es_response = es.search(
        index="prod-labq-documents-intfloat-multilingual-e5-large-instruct-hantaek",
        body=es_query
    )
    
    documents = []
    for hit in es_response["hits"]["hits"]:
        metadata = hit["_source"]["metadata"].copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
        
        documents.append({
            'text': hit["_source"]["text"],
            'metadata': {
                **metadata,
                'es_score': hit["_score"]
            }
        })
    print(f"Elasticsearch 검색 결과 수: {len(documents)}\n")
    
    # 2. FAISS 유사도 계산 (15개)
    print("2. FAISS 유사도 계산으로 필터링 (15개 문서)")
    texts = [doc['text'] for doc in documents]
    vectors = embedding_model.embed_documents(texts)
    query_vector = embedding_model.embed_query(query)
    
    similarities = np.dot(vectors, query_vector) / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query_vector))
    top_15_indices = np.argsort(similarities)[-15:][::-1]
    
    filtered_documents = [documents[i] for i in top_15_indices]
    for doc, sim in zip(filtered_documents, similarities[top_15_indices]):
        doc['metadata']['faiss_score'] = float(sim)
    print(f"FAISS 필터링 결과 수: {len(filtered_documents)}\n")
    
    # 3. Reranker (Top 5)
    print("3. Reranker로 Top 5 선정")
    pairs = [[query, doc['text']] for doc in filtered_documents]
    scores = reranker.predict(pairs)
    
    for doc, score in zip(filtered_documents, scores):
        doc['metadata']['reranker_score'] = score
    
    ranked_results = sorted(filtered_documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

In [40]:
# [셀 5] - ChromaDB 실험
def experiment_3(query: str):
    """ChromaDB -> Reranker 실험"""
    print("=== 실험 3: ChromaDB -> Reranker ===\n")
    
    # 1. ChromaDB 검색 (100개)
    print("1. ChromaDB 검색 (100개 문서)")
    langchain_chroma = Chroma(
        client=chroma_client,
        collection_name="financial_reports",
        embedding_function=embedding_model
    )
    base_retriever = langchain_chroma.as_retriever(search_kwargs={"k": 100})
    initial_docs = base_retriever.get_relevant_documents(query)
    print(f"ChromaDB 검색 결과 수: {len(initial_docs)}\n")
    
    # 2. Reranker (Top 5)
    print("2. Reranker로 Top 5 선정")
    pairs = [[query, doc.page_content] for doc in initial_docs]
    scores = reranker.predict(pairs)
    
    documents = []
    for doc, score in zip(initial_docs, scores):
        metadata = doc.metadata.copy()
        if 'base64_encoding' in metadata:
            metadata['has_base64'] = True
            del metadata['base64_encoding']
        if 'coordinates' in metadata:
            metadata['has_coordinates'] = True
            del metadata['coordinates']
            
        documents.append({
            'text': doc.page_content,
            'metadata': {
                **metadata,
                'chroma_score': doc.metadata.get('distance', 0),
                'reranker_score': score
            }
        })
    
    ranked_results = sorted(documents, key=lambda x: x['metadata']['reranker_score'], reverse=True)[:5]
    
    # 결과 출력
    print("\n=== Top 5 Documents ===")
    for i, doc in enumerate(ranked_results, 1):
        print(f"\n[Document {i}]")
        print("Metadata:")
        for key, value in doc['metadata'].items():
            if isinstance(value, float):
                print(f"- {key}: {value:.4f}")
            else:
                print(f"- {key}: {value}")
        print("Text:")
        print(doc['text'])
        print("-" * 100)
    
    return ranked_results

In [41]:
# [셀 6]은 기존거 사용
# [셀 7] - 테스트 실행
test_queries = [
    {
        "query": "DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?",
        "target_nid": 171,
        "difficulty": "easy"
    },
    {
        "query": "SK하이닉스에 대한 DS증권의 가장 최근 목표주가와 향후 2년간의 영업이익 전망은?",
        "target_nid": 171,
        "difficulty": "medium"
    },
    {
        "query": "메모리 반도체 업체 중 SK하이닉스의 향후 실적 전망은 어떠한가요? 특히 2024-2025년의 영업이익 추정치가 궁금합니다.",
        "target_nid": 171,
        "difficulty": "hard"
    }
]

# 실험 실행
results = run_experiments(test_queries)
analyze_results(results)


테스트 케이스: easy
Query: DS증권이 2024년 10월 25일에 발행한 SK하이닉스 관련 레포트에서 제시한 투자의견과 목표주가, 2024년 및 2025년 영업이익 전망치는 각각 얼마인가요?
=== 실험 1: Faiss -> ES -> Reranker ===

1. FAISS 검색 (100개 문서)
FAISS 검색 결과 수: 100

2. Elasticsearch 필터링 (15개 문서)
Elasticsearch 필터링 결과 수: 15

3. Reranker로 Top 5 선정

=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'



=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요<br>부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대<br>한다. 2025년 영업이익은 36조원으로 전망한다.</p>"}
- uuid: cf2e2bb8-5d26-4f46-8650-e330d0339abd
- nid: 171
- has_coordinates: True
- es_score: 24.7488
- faiss_score: 0.9458
- reranker_score: 0.99997925758

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'


FAISS 필터링 결과 수: 15

3. Reranker로 Top 5 선정

=== Top 5 Documents ===

[Document 1]
Metadata:
- category: paragraph
- page: 1
- id: 7
- company_name: SK하이닉스
- report_date: 20241025
- securities_firm: DS증권
- source_file: SK하이닉스_20241025_DS증권.pdf
- content_types: {'markdown': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'text': 'SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은\n8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요\n부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대\n한다. 2025년 영업이익은 36조원으로 전망한다.', 'html': "<br><p id='7' data-category='paragraph' style='font-size:16px'>SK하이닉스에 대한 투자의견과 목표주가 29만원을 유지한다. 4Q24 영업이익은<br>8.2조원, 2024년 영업이익은 24조원으로 전망한다. 1Q25 까지는 Non-AI 수요<br>부진이 지속될 전망이나 2H25 일반 서버 교체주기 도래에 따른 회복 역시 기대<br>한다. 2025년 영업이익은 36조원으로 전망한다.</p>"}
- uuid: cf2e2bb8-5d26-4f46-8650-e330d0339abd
- nid: 171
- has_coordinates: True
- es_score: 7.1608
- faiss_scor

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'



=== Top 5 Documents ===

[Document 1]
Metadata:
- category: list
- page: 1
- id: 2
- company_name: SK하이닉스
- report_date: 20241209
- securities_firm: 유진투자증권
- source_file: SK하이닉스_20241209_유진투자증권.pdf
- content_types: {'markdown': '- ◼ 최근 일련의 상황들을 보면 코리아 디스카운트의 가장 큰 요인은 어쩌면 대주주 리스크와 정치 지도자 리스크였을지도 모른\n- 다는 생각이 든다. 트럼프의 재등장으로 글로벌 지경학은 밀림의 한 가운데 들어섰다. 거기에 더해 반도체 업황은 둔화하고\n- 있으며, 수출 통제 등 부담까지 더해지고 있다. 하지만 이런 절체절명의 상황에서 우리나라는 계엄 발동과 해제, 그리고\n- 지도자 공백이라는 초현실적 상황을 맞이하게 되었다. 이와 같은 국내외적 리스크를 감안할 때 주요 기업들의 실적 전망\n- 하향과 밸류에이션 조정이 불가피해 보인다.\n- ◼ IT 수요 부진과 메모리 재고조정 영향, 반도체 규제 등을 반영해 SK하이닉스의 실적을 다음과 같이 조정한다. 4분기\n- 실적은 매출 18.4조원, 영업이익 7.3조원을 예상한다. 인상적인 기술력을 감안할 때 2025년 실적은 매출 70.4조원\n- 영업이익 25.6조원으로 성장세를 이어갈 가능성은 있지만, 현재 시장 기대는 다소 높아 보인다.\n- ◼ 밸류업을 가로막는 코리아 리스크로 타겟 밸류에이션을 P/B 2.0에서 1.8배로 조정하고, 목표주가를 220,000원으로\n- 하향한다. 다만, 현주가는 P/B 1.4배로 타겟 대비 현저히 낮다는 점에서 투자의견은 ‘BUY’를 유지한다.\n', 'text': '◼ 최근 일련의 상황들을 보면 코리아 디스카운트의 가장 큰 요인은 어쩌면 대주주 리스크와 정치 지도자 리스크였을지도 모른\n다는 생각이 든다. 트럼프의 재등장으로 글로벌 지경학은 밀림의 한 가운데 들어섰다.

## 최종 결과

- 성능 대비 처리 속도 등을 종합적으로 고려했을 때, 아래와 같이 설정
- Faiss 100개
- ES 15개
